In [2]:
from keras.layers import Input, Dense, Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K
from keras.datasets import mnist
from keras.callbacks import TensorBoard
import numpy as np
import pickle
from datetime import datetime

(x_train, _), (x_test, _) = mnist.load_data()

In [3]:
RUN_ON_MNIST = False 
L = 4

# Notes:
* Add `Input` argument `channels_first` when using Z3 data and 
  change the input dimensions.
* You will have to manualy add periodic edges for periodic padding before real runn
* **requires** running the transform notebook to get the data into a format for the auto encoder

In [8]:
if RUN_ON_MNIST:
    input_obj = Input(shape=(28, 28, 1))
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_obj)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)
    
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)  

else:
    input_obj = Input(shape=(L*2, L*2, 1))
    x = Conv2D(16, (3, 3), strides=2, activation='relu', padding='same', use_bias=True)(input_obj)
    encoded = Conv2D(32, (3, 3), strides=2, activation='relu', padding='same', use_bias=True)(x)
    x = Conv2DTranspose(16, (3, 3), strides=2, activation='relu', padding='same', use_bias=True)(encoded)
    decoded = Conv2DTranspose(1, (3, 3), strides=2, activation='relu', padding='same', use_bias=True)(x)

autoencoder = Model(input_obj, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [9]:
autoencoder.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 8, 8, 1)]         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 4, 4, 16)          160       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 2, 2, 32)          4640      
_________________________________________________________________
conv2d_transpose_4 (Conv2DTr (None, 4, 4, 16)          4624      
_________________________________________________________________
conv2d_transpose_5 (Conv2DTr (None, 8, 8, 1)           145       
Total params: 9,569
Trainable params: 9,569
Non-trainable params: 0
_________________________________________________________________


In [5]:
if RUN_ON_MNIST:
    x_train = x_train.astype('float32') / 255.
    x_test = x_test.astype('float32') / 255.
    x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))  # adapt this if using `channels_first` image data format
    x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))  # adapt this if using `channels_first` image data format
else:
    with open('pickle_data.pkl', 'rb') as f:
        all_data = pickle.load(f)
    all_data = all_data.astype('float32') / 5.0
    n_records = len(all_data)
    x_train = all_data[: n_records - int(n_records/4)]
    x_test = all_data[int(n_records/4) :]
    x_train = np.reshape(x_train, (len(x_train), L * 2, L * 2, 1))
    x_test = np.reshape(x_test, (len(x_test), L * 2, L * 2, 1))

In [6]:
x_train.shape

(7500, 8, 8, 1)

In [8]:
autoencoder.fit(x_train, x_train,
                epochs=120,
                batch_size=90,
                shuffle=True,
                validation_data=(x_test, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/autoencoder/{}'.format(datetime.now()))])

Epoch 1/120
84/84 [==============================] - 1s 8ms/step - loss: 0.5879 - val_loss: 0.5855
Epoch 2/120
84/84 [==============================] - 1s 7ms/step - loss: 0.5826 - val_loss: 0.5780
Epoch 3/120
84/84 [==============================] - 1s 7ms/step - loss: 0.5735 - val_loss: 0.5685
Epoch 4/120
84/84 [==============================] - 1s 8ms/step - loss: 0.5640 - val_loss: 0.5587
Epoch 5/120
84/84 [==============================] - 1s 8ms/step - loss: 0.5573 - val_loss: 0.5554
Epoch 6/120
84/84 [==============================] - 1s 8ms/step - loss: 0.5539 - val_loss: 0.5513
Epoch 7/120
84/84 [==============================] - 1s 7ms/step - loss: 0.5489 - val_loss: 0.5455
Epoch 8/120
84/84 [==============================] - 1s 8ms/step - loss: 0.5431 - val_loss: 0.5396
Epoch 9/120
84/84 [==============================] - 1s 8ms/step - loss: 0.5379 - val_loss: 0.5350
Epoch 10/120
84/84 [==============================] - 1s 8ms/step - loss: 0.5326 - val_loss: 0.5300
Epoch 11/